In [1]:
import sys
import os
from os.path import exists
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt

sys.path.insert(1,'/data1/gridsan/groups/manthiram_lab/Utils')
sys.path.insert(1,'/data1/gridsan/groups/manthiram_lab/QMkit')
from dft_utils import checkvibs, check, no_to_symbol
from gen_3D import write_xyz_from_xyz_arr,combine_labels_and_xyz

import re
import importlib
import cclib
import plot as pl
import pathlib
from tqdm import tqdm
from datetime import datetime, timedelta
from rdkit import Chem

ImportError: cannot import name 'write_xyz_from_xyz_arr' from 'gen_3D' (/data1/gridsan/groups/manthiram_lab/QMkit/gen_3D.py)

In [2]:
importlib.reload(pl)
importlib.reload(utils)
importlib.reload(convertxyz)

<module 'convertxyz' from '/data1/gridsan/groups/manthiram_lab/DFT/DFT_script_gen/convertxyz.py'>

In [3]:
#Define Current directory
cwd= '/home/gridsan/jmaalouf/vHTP/Code/ylide/Library'

# Load Data

In [4]:
df=pd.read_csv('../sheets/Ylides_YlideRads_Cleaned.csv')
y=df['Ylides'].to_list()
yr=df['Ylides Rad'].to_list()
yh=df['Ylides H'].to_list()

## Get Num Atoms

In [10]:
num_atoms=[]
num_atoms_total=[]
for i, s in enumerate(tqdm(y)):
    mol=Chem.MolFromSmiles(s)
    num_atoms.append(mol.GetNumAtoms())
    mol=Chem.AddHs(mol)
    num_atoms_total.append(mol.GetNumAtoms())

100%|██████████| 19097/19097 [00:01<00:00, 9843.39it/s]


In [8]:
with open(f"../pickled_data/num_atoms.pkl", "wb") as file: 
    pickle.dump(num_atoms, file)
    
with open(f"../pickled_data/num_atoms_total.pkl", "wb") as file: 
    pickle.dump(num_atoms_total, file)

# Check for Failed Calcs

In [5]:
trueChoices = ['true', 't','T']
falseChoices = ['false', 'f','F']

inp=input("Do you want to resubmit the memory allocation errors? (T/F)").lower()

assert inp in trueChoices or inp in falseChoices
if inp in trueChoices:
    resubmit=True
elif inp in falseChoices:
    resubmit=False

orig_dir='/home/gridsan/groups/manthiram_lab/vHTP/Code/ylide/Library'

failed_dict={}
geom_failed_dict={}

functional='M062X'
basis= 'Def2SVPD'
solvorgas='solv'
solvmethod='SMD'
solvent='acetonitrile'

if solvorgas== 'gas':
    ylide_type=[['ylide',''],['ylide_rad','_y_h_opt'],['ylide_h','']]
elif solvorgas =='solv':
    ylide_type=[['ylide','_gas_preopt'],['ylide_rad','_gas_preopt'],['ylide_h','_gas_preopt']]
    #ylide_type=[['ylide','_gas_preopt'],['ylide_h','_gas_preopt']]

start=0
stop=10000
n=stop-start
error_occurred=False

for v,y_list in enumerate(ylide_type):
    
    y=y_list[0]
    sub_suff=y_list[1]
    
    failed_dict[f'{y}{sub_suff}']=[]
    geom_failed_dict[f'{y}{sub_suff}']=[]
    
    for i in tqdm(range(start,stop)):
        n_str=str(i).zfill(7)
        
        
        assert solvorgas == 'gas' or solvorgas=='solv','SOLVORGAS NOT SET TO A VALID VALUE'
        
        if solvorgas=='gas':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs', str(i).zfill(7) , y, functional, basis,'gas', f'{str(i).zfill(7)}_{y}{sub_suff}.log'))
            
        elif solvorgas=='solv':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs',str(i).zfill(7) , y, functional, basis,solvent,solvmethod, f'{str(i).zfill(7)}_{y}{sub_suff}.log'))
        
        par_path=path.parent.absolute()
        
        if exists(path):
            
            if check(path, 'Normal termination of Gaussian', count=2)==False:
                error_occurred=True
                
                if check(str(path),'galloc:  could not allocate memory.'):
                    
                    #print(f'MEMRORY ALLOCATION ERROR WITH {y}{sub_suff} MOLECULE {i}')
                    failed_dict[f'{y}{sub_suff}'].append(i)
                    
                    if resubmit:
                        os.chdir(par_path)
                        subprocess.run(f'sbatch {n_str}_{y}{sub_suff}.sh',shell=True)
                        os.chdir(orig_dir)
                                
                elif check(str(path),'Converged',120):
                    #print(f'GEOMETRY OPT ERROR WITH {y}{sub_suff} MOLECULE {i}')
                    #failed_dict[f'{y}{sub_suff}'].append(i)
                    geom_failed_dict[f'{y}{sub_suff}'].append(i)
                    
                else:
                    #Try one more time to extract the data because there could still be some optimized geometry
                    #steps that can be extracted
                    #print(f'UNKNOWN ERROR, {y}{sub_suff} MOLECULE {i} CALCULATION DID NOT FINSIH')
                    failed_dict[f'{y}{sub_suff}'].append(i)
                    
                    if resubmit:
                        os.chdir(par_path)
                        #subprocess.run(f'sbatch {n_str}_{y}{sub_suff}.sh',shell=True)
                        os.chdir(orig_dir)
        else:
            #print(f'{y} MOLECULE {i}{sub_suff} HAS NOT BEEN SUBMITTED')
            failed_dict[f'{y}{sub_suff}'].append(i)
            
            if resubmit:
                os.chdir(par_path)
                subprocess.run(f'sbatch {n_str}_{y}{sub_suff}.sh',shell=True)
                os.chdir(orig_dir)
                
    if error_occurred==False:
        print(f'NO ERRORS WITH {y}{sub_suff}')
        
        
    error_occurred=False
    print('')
    


Do you want to resubmit the memory allocation errors? (T/F) f


  0%|          | 2/10000 [00:00<12:57, 12.86it/s]

  0%|          | 2/10000 [00:00<10:15, 16.23it/s]

100%|██████████| 10000/10000 [18:59<00:00,  8.78it/s]  

### Save Dictionaries

In [9]:
with open(f"/data1/groups/manthiram_lab/vHTP/Code/ylide/Library/python_scripts/dicts/failed_dict_{solvorgas}_{'0817'}.pkl", "wb") as file: 
    pickle.dump(failed_dict, file)

In [9]:
#os.chdir(cwd)
with open(f"/data1/groups/manthiram_lab/vHTP/Code/ylide/Library/python_scripts/dicts/failed_dict_{solvorgas}_{'0626'}.pkl", "wb") as file: 
    pickle.dump(failed_dict, file)

In [ ]:
with open(f"../pickled_data/failed_dict_{solvorgas}.pkl", "wb") as file: 
    pickle.dump(failed_dict, file)

In [22]:
with open(f"../pickled_data/geom_failed_dict_{solvorgas}.pkl", "wb") as file: 
    pickle.dump(geom_failed_dict, file)

#### Visualize contents of dictionaries

In [7]:
for key, value in failed_dict.items():
    print(f'{key} has {len(value)} failures and a fail percentage of {len(value)/n*100}%')

ylide_gas_preopt has 15 failures and a fail percentage of 0.15%
ylide_rad_gas_preopt has 1609 failures and a fail percentage of 16.09%
ylide_h_gas_preopt has 572 failures and a fail percentage of 5.72%


In [8]:
for key, value in geom_failed_dict.items():
    print(f'{key} has {len(value)} failures and a fail percentage of {len(value)/n*100}%')

ylide_gas_preopt has 129 failures and a fail percentage of 1.29%
ylide_rad_gas_preopt has 231 failures and a fail percentage of 2.31%
ylide_h_gas_preopt has 301 failures and a fail percentage of 3.01%


# Extract Data from Directories

In [27]:
functional='M062X'
basis= 'Def2TZVP'
solvorgas='gas'
solvmethod='SMD'


orig_dir='/home/gridsan/groups/manthiram_lab/vHTP/Code/ylide/Library'
if solvorgas=='gas':
    solvent='gas'
    ylide_type=[['ylide',''],['ylide_rad','_y_h_opt'],['ylide_h','']]
    #ylide_type=[['ylide_h','']]
else:
    solvent='acetonitrile'
    ylide_type=[['ylide','_gas_preopt'],['ylide_rad','_gas_preopt'],['ylide_h','_gas_preopt'] ]

start=1335
stop=1336
n=stop-start
data=np.empty((n,10))
atom_coords=[] #Dim1= Ylide Dim2=Ylide Rad Dim3= Ylide_H
atom_labels=[] #Dim1= Ylide Dim2=Ylide Rad Dim3= Ylide_H
wall_time=[] #Calculation time in seconds

for v,y_list in enumerate(ylide_type):
    
    y=y_list[0]
    sub_suff=y_list[1]
    
    temp_atom_coords=[]
    temp_atom_labels=[]
    temp_wall_time=[]
    
    for k,i in enumerate(tqdm(range(start,stop))):
        n_str=str(i).zfill(7)

        
        
        assert solvorgas == 'gas' or solvorgas=='solv','SOLVORGAS NOT SET TO A VALID VALUE'
        
        if solvorgas=='gas':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs', str(i).zfill(7) , y, functional, basis,'gas', f'{str(i).zfill(7)}_{y}{sub_suff}.log'))
            
        elif solvorgas=='solv':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs',str(i).zfill(7) , y, functional, basis,solvent,solvmethod, f'{str(i).zfill(7)}_{y}{sub_suff}.log'))
        
        
        
        if exists(path): #If calculation was run
            
            if check(path, 'Normal termination of Gaussian',count=2): #If calculation was run successfully
                try:
                    
                    temp_data=cclib.io.ccread(str(path))

                    meta_data=temp_data.metadata
                    data[k,2*v]=temp_data.enthalpy
                    data[k,2*v+1]=temp_data.freeenergy
                    data[k,6]=checkvibs(temp_data.vibfreqs,asint=True)
                    temp_atom_coords.append(temp_data.atomcoords[-1,:,:])
                    temp_atom_labels.append([no_to_symbol(x) for x in temp_data.atomnos])
                    temp_wall_time.append(sum(meta_data['wall_time'][i].seconds for i in range(len(meta_data['wall_time']))))
                    
                    continue
                except Exception as e:
                    exc_type, exc_obj, exc_tb = sys.exc_info()
                    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                    print(e)       
                    print(exc_type, fname, exc_tb.tb_lineno)

                    data[k,2*v]= 1
                    data[k,2*v+1]= 1
                    data[k,6]=-1
                    temp_atom_coords.append([])
                    temp_atom_labels.append([])
                    temp_wall_time.append(0)
                    continue
            else: #Run unseccessfully
                
                #Check if geometry did not optimize
                if check(str(path),'Converged',120):
                    
                    temp_data=cclib.io.ccread(str(path))
                    meta_data=temp_data.metadata
                    data[k,2*v]=1
                    data[k,2*v+1]=1
                    data[k,6]=1
                    temp_atom_coords.append(temp_data.atomcoords[-1,:,:])
                    temp_atom_labels.append([no_to_symbol(x) for x in temp_data.atomnos])
                    temp_wall_time.append(0)
                    continue
                
                else:
                    data[k,2*v]= 1
                    data[k,2*v+1]= 1
                    data[k,6]=-1
                    temp_atom_coords.append([])
                    temp_atom_labels.append([])
                    temp_wall_time.append(0)
                    continue
        else:
            data[k,2*v]= 1
            data[k,2*v+1]= 1
            data[k,6]=-1
            temp_atom_coords.append([])
            temp_atom_labels.append([])
            temp_wall_time.append(0)
            continue

    atom_coords.append(temp_atom_coords)
    atom_labels.append(temp_atom_labels)
    wall_time.append(temp_wall_time)

100%|██████████| 1/1 [00:00<00:00,  3.94it/s]


In [28]:
tmp=combine_labels_and_xyz(atom_labels[2][0],atom_coords[2][0])
write_xyz_from_xyz_arr(tmp,'temp',include_total_atoms=True)


# Get Atom Labels & Coords

In [96]:
functional='M062X'
basis= 'Def2TZVP'
solvorgas='gas'
solvmethod='SMD'

orig_dir='/home/gridsan/groups/manthiram_lab/vHTP/Code/ylide/Library'
if solvorgas=='gas':
    solvent=='gas'
    ylide_type=[['ylide',''],['ylide_rad','_y_h_opt'],['ylide_h','']]
    #ylide_type=[['ylide_h','']]
else:
    solvent='acetonitrile'
    ylide_type=[['ylide','_gas_preopt'],['ylide_rad','_gas_preopt'],['ylide_h','_gas_preopt'] ]

start=0
stop=2
n=stop-start
data=np.empty((n,10))
atom_coords=[] #Dim1= Ylide Dim2=Ylide Rad Dim3= Ylide_H
atom_labels=[] #Dim1= Ylide Dim2=Ylide Rad Dim3= Ylide_H
wall_time=[] #Calculation time in seconds

for v,y_list in enumerate(ylide_type):
    
    y=y_list[0]
    sub_suff=y_list[1]
    
    temp_atom_coords=[]
    temp_atom_labels=[]
    temp_wall_time=[]
    
    for k,i in enumerate(tqdm(range(start,stop))):
        n_str=str(i).zfill(7)

        
        
        assert solvorgas == 'gas' or solvorgas=='solv','SOLVORGAS NOT SET TO A VALID VALUE'
        
        if solvorgas=='gas':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs', str(i).zfill(7) , y, functional, basis,'gas', f'{str(i).zfill(7)}_{y}{sub_suff}.xyz'))
            
        elif solvorgas=='solv':
            path=pathlib.Path(os.path.join(orig_dir, 'Calcs',str(i).zfill(7) , y, functional, basis,solvent,solvmethod, f'{str(i).zfill(7)}_{y}{sub_suff}.xyz'))
        
        
        
        if exists(path): #If calculation was run
            
            try:
                temp_temp_atom_labels=[]
                temp_temp_atom_coords=[]
                
                with open(path) as fp:
                    Lines = fp.readlines()
                    for line in Lines:
                        l=line.strip().split()
                        if len(l)>0:
                            temp_temp_atom_labels.append(l[0])
                            temp_temp_atom_coords.append(l[1:])
                temp_atom_labels.append(temp_temp_atom_labels)
                temp_atom_coords.append(np.array(temp_temp_atom_coords,dtype='float'))

                #temp_data=cclib.io.ccread(str(path))

#                     meta_data=temp_data.metadata
#                     data[k,2*v]=temp_data.enthalpy
#                     data[k,2*v+1]=temp_data.freeenergy
#                     data[k,6]=checkvibs(temp_data.vibfreqs,asint=True)
#                     temp_atom_coords.append(temp_data.atomcoords[-1,:,:])
                #temp_atom_labels.append([no_to_symbol(x) for x in temp_data.atomnos])
#                     temp_wall_time.append(sum(meta_data['wall_time'][i].seconds for i in range(len(meta_data['wall_time']))))
                continue
            except Exception as e:
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                print(e)       
                print(exc_type, fname, exc_tb.tb_lineno)

                data[k,2*v]= 1
                data[k,2*v+1]= 1
                data[k,6]=-1
                temp_atom_coords.append([])
                temp_atom_labels.append([])
                temp_wall_time.append(0)
                continue

        else:
            data[k,2*v]= 1
            data[k,2*v+1]= 1
            data[k,6]=-1
            temp_atom_coords.append([])
            temp_atom_labels.append([])
            temp_wall_time.append(0)
            continue

    atom_coords.append(temp_atom_coords)
    atom_labels.append(temp_atom_labels)
#     wall_time.append(temp_wall_time)

100%|██████████| 2/2 [00:00<00:00, 1314.21it/s]


In [70]:
with open(f"../pickled_data/atom_labels.pkl","wb") as file:
    pickle.dump(atom_labels, file)

In [19]:
np.mean(wall_time[2][:])

1876.582

#### Wall Times For Different Triples Conditons

[5,1,1] wall_time: [1816, 2098, 2524, 2864, 1405, 1256, 1628, 1559, 1741, 1783] <br>
[10,1,2] wall_time: [[1800, 0, 0, 0, 1398, 1248, 1618, 1560, 1732, 1787]] <br>
[5,2,1] wall_time: [3784, 0, 5337, 0, 0, 2829, 0, 3480, 3763, 0] <br>

There seems to be no effect from number of threads, it seems that we still have an effect from RAM and it scales pretty linearly so I will use 1 node per calculation

In [20]:
print('CALCULATING REDOX POTENTIALS')    
for i, v in enumerate(tqdm(range(np.shape(data)[0]))):
    data[i,7]=calcE(data[i,3],data[i,1])
    
    
print('CALCULATING DPFE')
for i, v in enumerate(tqdm(range(np.shape(data)[0]))):
    data[i,8]=calcDPFE(data[i,5],data[i,1],solvent=solvent)

print('CALCULATING GHBind ')
for i, v in enumerate(tqdm(range(np.shape(data)[0]))):
    data[i,9]=calcGHBind(data[i,5],data[i,3],solvent='gas')
    

100%|██████████| 1000/1000 [00:00<00:00, 703270.29it/s]

CALCULATING REDOX POTENTIALS
CALCULATING DPFE
CALCULATING GHBind 


In [12]:
!pwd

/home/gridsan/groups/manthiram_lab/vHTP/Code/ylide/Library


In [21]:
#Save Extracted data
cwd='/home/gridsan/jmaalouf/vHTP/Code/ylide/Library'
os.chdir(cwd)

with open(f"../pickled_data/atom_labels.pkl","wb") as file:
    pickle.dump(atom_labels, file)

with open(f"pickled_data/wall_time_{solvorgas}.pkl", "wb") as file: 
    pickle.dump(wall_time, file)

with open(f"pickled_data/atom_coords_{solvorgas}.pkl", "wb") as file: 
    pickle.dump(atom_coords, file)


np.save(f'pickled_data/data_{solvorgas}.npy', data)


In [58]:
df=pd.DataFrame(data, columns=['ylide_enthalpy','ylide_freeenergy','ylide_rad_enthalpy','ylide_rad_freeenergy','ylide_h_enthalpy','ylide_h_freeenergy','Vibs All Positivse','Redox Potential (V vs SHE)','Deprotonation Free Energy (kJ/mol)'])